## Voice Cloning System (VC) with TTS

### Data downloading and preprocessing

In [4]:
import os
import glob
import librosa
import numpy as np
from TTS.api import TTS
import soundfile as sf
import jiwer

In [5]:
# Step 1: Data Preparation
def download_and_prepare_timit(timit_path='TIMIT'):
    audio_paths_train= glob.glob(os.path.join(timit_path, 'TRAIN/DR*/**/*.WAV'), recursive=True)
    #print(audio_paths_train)
    audio_paths_test= glob.glob(os.path.join(timit_path, 'TEST/DR*/**/*.WAV'), recursive=True)
    transcription_paths_train = glob.glob(os.path.join(timit_path, 'TRAIN/DR*/**/*.TXT'), recursive=True)
    transcription_paths_test = glob.glob(os.path.join(timit_path, 'TEST/DR*/**/*.TXT'), recursive=True)

    audio_files_train = []
    transcriptions_train = []
    
    for audio_path in audio_paths_train:
        transcription_path_train = audio_path.replace(".WAV", ".TXT")
        if os.path.exists(transcription_path_train):
            with open(transcription_path_train, 'r') as f:
                transcription = f.readlines()[0].strip().split(' ', 2)[-1]
                audio_files_train.append(audio_path)
                transcriptions_train.append(transcription)
    
    audio_files_test = []
    transcriptions_test = []
    
    for audio_path in audio_paths_test:
        transcription_path_test = audio_path.replace(".WAV", ".TXT")
        if os.path.exists(transcription_path_test):
            with open(transcription_path_test, 'r') as f:
                transcription = f.readlines()[0].strip().split(' ', 2)[-1]
                audio_files_test.append(audio_path)
                transcriptions_test.append(transcription)
                
    return audio_files_train, transcriptions_train,audio_files_test, transcriptions_test 

# Usage
timit_path = 'TIMIT\\data'
audio_files_train, transcriptions_train,audio_files_test, transcriptions_test = download_and_prepare_timit(timit_path)

In [6]:
# Step 2: Feature Extraction
def extract_mel_spectrogram(audio_path, sr=22050, n_mels=80):
    y, sr = librosa.load(audio_path, sr=sr)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    return mel_spec_db

mel_train = [extract_mel_spectrogram(f) for f in audio_files_train]
mel_test = [extract_mel_spectrogram(f) for f in audio_files_test]

### Model loading

In [15]:

modelname = TTS().listmodels()[0]
# Step 3: Load Pre-trained Models
# Initialize Tacotron 2 model
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=True, gpu=False)

# Initialize HiFi-GAN vocoder
vocoder = TTS(model_name="vocoder_models/en/ljspeech/hifigan_v2", progress_bar=True, gpu=False)

FileNotFoundError: [WinError 2] The system cannot find the file specified